In [188]:
import h5py
import numpy as np

os.environ['HDF5_USE_FILE_LOCKING'] = 'TRUE'

def GetSize(path):
    file = h5py.File(path, mode='r')
    n = file["times"].shape[0]
    file.close()
    return n

def Load(path, chunk=[]):
    # @brief Reads grid data and time data from a specified hdf5 file.
    # @param path The path of the HDF5 file
    # @param dataset A string name of the main dataset containing the states data
    # @param times A string name of the time dataset containing all time points
    # @param chunk An optional [start, end] list that specifies how many time steps to load
    file = h5py.File(path, "r")
    if chunk:
        data_array = np.array(file['data'][chunk[0]:chunk[1],0,:,:])
        time_array = np.array(file['times'][chunk[0]:chunk[1]])
    else:
        data_array = np.array(file['data'][:, 0, :, :])
        time_array = np.array(file['times'])
    file.close()
    return data_array #, time_array)

ref_ = 'HLLE'
reconstruction = 'Characteristics'

f = 2
bnx = 100
bny = 70

path = '/srv/public/Maikel/FiniteVolumeSolver/build_2D-Release/ReferenceData/SlopedChannel_{}_{}x{}-1.h5'.format(reconstruction, bnx*f, bny*f)
ref_path = '/srv/public/Maikel/FiniteVolumeSolver/build_2D-Release/ReferenceData/SlopedChannel_{}_{}x{}-1.h5'.format(ref_, bnx*f, bny*f)
file = h5py.File(path, mode='r')
data = np.array(file['data'][:, 0, :, :])
datau = np.array(file['data'][:, 1, :, :])
times = np.array(file['times'])
dx = file['data'].attrs['cell_size']
xlower = file['data'].attrs['xlower']
file.close()

file = h5py.File(ref_path, mode='r')
ref_data = np.array(file['data'][:, 0, :, :])
file.close()

In [189]:
print(data.shape)
print(ref_data.shape)

(16, 200, 140)
(1, 200, 140)


In [187]:
import matplotlib.pyplot as plt
import os


nx = [data.shape[1], data.shape[2]]
xupper = xlower + dx * nx

vmin = None #0.4
vmax = None #2.0

outpath = '/srv/public/Maikel/FiniteVolumeSolver/extra/{}'.format(reconstruction)
os.makedirs(outpath, exist_ok=True)


for n in range(data.shape[0]):
# for n in range(1):
    rho = np.array(data[n,:,:]) # np.abs(np.array(data[n,:,:]) - ref_data)
    # drho_dx = np.gradient(rho, 0)
    # drho = np.sqrt(drho_dx * drho_dx + drho_dy * drho_dy)
    # u = rhou / rho
    nticks = 5

    real_x = np.around(np.linspace(xlower[0], xupper[0], nticks, endpoint=True), 3)
    real_y = np.around(np.linspace(xlower[1], xupper[1], nticks, endpoint=True), 3)

    dxt = [int(nx[0] / (nticks - 1)), int(nx[1] / (nticks - 1))]
    xticks = list(range(0, nx[0] + 1, dxt[0]))
    yticks = list(range(0, nx[1] + 1, dxt[1]))
    
    imdata = np.log(1.0 + np.reshape(rho, [nx[1], nx[0]]))
    f, ax = plt.subplots()
    if vmin is not None:
        ax.imshow(imdata, origin='lower', vmin=vmin, vmax=vmax, interpolation=None)
    else:
        ax.imshow(imdata, origin='lower', cmap='jet', interpolation='none')
    ax.set_xticks(xticks)
    ax.set_xticklabels(real_x)
    ax.set_yticks(yticks)
    ax.set_yticklabels(real_y)
    ax.set_title('Time: {:.3}s'.format(times[n]))
    f.savefig('{}/Density_{}.png'.format(outpath, n))
    f.clf()

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

...

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

In [177]:

last_step = data[data.shape[0] - 1, :, :]
err_inf = np.max(np.abs(ref_data - last_step))
err_1 = np.sum(np.abs(ref_data - last_step)) / ref_data.size

print(err_inf)
print(err_1)

0.23366024478235503
0.005349322291493321


In [190]:

last_step = data[data.shape[0] - 1, :, :]
err_inf = np.max(np.abs(ref_data - last_step))
err_1 = np.sum(np.abs(ref_data - last_step)) / ref_data.size

print(err_inf)
print(err_1)

0.10254106221075121
0.0013089504458987256


In [185]:

last_step = data[data.shape[0] - 1, :, :]
err_inf = np.max(np.abs(ref_data - last_step))
err_1 = np.sum(np.abs(ref_data - last_step)) / ref_data.size

print(err_inf)
print(err_1)


1.5647504254993634
0.004927912927297162


In [172]:
print(0.2314754926480802 / 0.10010925681177296 / 2.0)
print(0.10010925681177296 / 0.042826102710879166 / 2.0)

1.1561143295834477
1.1687878475379232


In [173]:

print(0.005336369011994841 / 0.0013199313990917467 / 2.0)
print(0.005336369011994841 / 0.00034935667824376285 / 8.0)

2.021456954379156
1.909355589973652


In [174]:
print(1.97e-2 / 6.19e-3 / 2.0)
print(6.19e-3 / 1.72e-3 / 2.0)

1.591276252019386
1.7994186046511629


In [175]:
print(2.63e-1 / 1.32e-1 / 2.0)
print(1.32e-1 / 6.09e-2 / 2.0)

0.9962121212121212
1.083743842364532
